# Plenary Protocols

In [1]:
from pathlib import Path
import glob
import numpy as np
import pandas as pd

In [2]:
VERBOSE = False

In [3]:
def show(table):
    with pd.option_context("display.max_colwidth", None):
        display(table)


def preview_lines(filepath, N=5):
    with open(filepath) as temp:
        head = [next(temp) for i in range(N)]
    temp.close()
    return head

### Data Loading
All csv-files in the specified directory will be parsed and concatenated.

In [4]:
path = "plpr-scraper/data/out"
all_files = sorted(glob.glob(path + "/*.csv"))

df_list = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, escapechar="\\")
    df_list.append(df)

df = pd.concat(df_list, axis=0, ignore_index=True)
df_prior_shape = df.shape
show(df[:10])

,id,sitzung,wahlperiode,in_writing,speaker,speaker_cleaned,speaker_fp,sequence,text,filename,type,speaker_party
0,2323997,1,17,0,Alterspräsident Dr. Heinz Riesenhuber,Dr. Heinz Riesenhuber,heinz-riesenhuber,0,"Guten Morgen, meine sehr verehrten Damen und Herren! Liebe Kolleginnen und Kollegen, ich begrüße Sie zur konstituierenden Sitzung des 17. Deutschen Bundestags.\n\nParlamentarischer Brauch ist es - das entspricht unserer Geschäftsordnung; ich kann die Paragrafen zitieren -, dass der Älteste die erste Sitzung des Bundestags eröffnet. Ich bin am Sonntag, dem 1. Dezember 1935, geboren. Wenn jemand von den Kollegen im Saal älter ist als ich, dann spreche er jetzt oder er schweige für immer.",data/txt/17001.txt,chair,NaN
1,2323998,1,17,0,NaN,NaN,NaN,1,Heiterkeit und Beifall,data/txt/17001.txt,poi,NaN
2,2323999,1,17,0,Alterspräsident Dr. Heinz Riesenhuber,Dr. Heinz Riesenhuber,heinz-riesenhuber,2,"Unser Präsident würde sagen: Ich höre und sehe keinen Widerspruch.\n\nMeine Damen und Herren, damit rufe ich Punkt 1 der Tagesordnung auf:\n\nEröffnung der Sitzung durch den Alterspräsidenten\n\nIch eröffne die erste Sitzung in der 17. Wahlperiode.\n\nIch begrüße den Herrn Bundespräsidenten. Wir freuen uns, Herr Bundespräsident, dass Sie wieder bei uns sind.",data/txt/17001.txt,chair,NaN
3,2324000,1,17,0,NaN,NaN,NaN,3,Beifall,data/txt/17001.txt,poi,NaN
4,2324001,1,17,0,Alterspräsident Dr. Heinz Riesenhuber,Dr. Heinz Riesenhuber,heinz-riesenhuber,4,"Ich begrüße herzlich die ehemalige Präsidentin des Deutschen Bundestages, Frau Dr. Rita Süssmuth.",data/txt/17001.txt,chair,NaN
5,2324002,1,17,0,NaN,NaN,NaN,5,Beifall,data/txt/17001.txt,poi,NaN
6,2324003,1,17,0,Alterspräsident Dr. Heinz Riesenhuber,Dr. Heinz Riesenhuber,heinz-riesenhuber,6,"Sie haben uns mit Würde und Klugheit über die Jahre geführt - die Verbundenheit bleibt.\n\nIch habe die Freude, Botschafter und Missionschefs zahlreicher Staaten hier zu begrüßen. Sie alle sind herzlich willkommen - in der Verbundenheit der Gemeinschaft der Völker.",data/txt/17001.txt,chair,NaN
7,2324004,1,17,0,NaN,NaN,NaN,7,Beifall,data/txt/17001.txt,poi,NaN
8,2324005,1,17,0,Alterspräsident Dr. Heinz Riesenhuber,Dr. Heinz Riesenhuber,heinz-riesenhuber,8,"Ich darf einen einzigen Kollegen besonders begrüßen, weil er heute mit uns seinen Geburtstag feiert, die schönste Party, die man sich vorstellen kann. Es ist der Kollege Henning Otte, dem ich herzlich gratuliere.",data/txt/17001.txt,chair,NaN
9,2324006,1,17,0,NaN,NaN,NaN,9,Beifall,data/txt/17001.txt,poi,NaN


### Data Cleansing
Quick look at the data before cleansing with Pandas-Profiling. We've got 592199 observations, many of which are not speeches but contributions by the *chair* (president and vice-presidents of the parliament) and *POIs*, points of information (interruptions or interjections by MPs other than the speaker or chair). Speeches span multiple rows and require concatenation.

In [5]:
from pandas_profiling import ProfileReport

profile = ProfileReport(df, title="Pre-Cleansing Pandas Profiling Report")

In [6]:
if VERBOSE:
    profile.to_notebook_iframe()

For now, we'll keep only speeches and replace newlines read-in literally.

In [7]:
df.dropna(subset=["text"], inplace=True)
df.loc[:, "text"] = df.loc[:, "text"].str.replace("\n\n", " ").replace("\n", " ")
df.loc[:, "text"] = df.loc[:, "text"].astype(str)
df = df[df["type"] == "speech"].copy()
df.reset_index(drop=True, inplace=True)
df.drop(["Unnamed: 0", "row.names"], axis=1, inplace=True, errors="ignore")
print(f"The shape is reduced from {df_prior_shape[0]} rows to {df.shape[0]}")
show(df[["speaker_cleaned", "text"]][:3])

The shape is reduced from 592199 rows to 238648


,speaker_cleaned,text
0,Volker Kauder,"Herr Alterspräsident, ich schlage für die CDU/CSU-Bundestagsfraktion den Kollegen Dr. Norbert Lammert vor."
1,Dr. Norbert Lammert,"Herr Präsident, ich nehme die Wahl gerne an."
2,Gerda Hasselfeldt,"Herr Präsident, ich nehme die Wahl gerne an und bedanke mich herzlich für das Vertrauen."


todo describe what happens

In [8]:
df["previous_speaker_fp"] = df["speaker_fp"].shift(1)
df["new_speaker"] = df["speaker_fp"] != df["previous_speaker_fp"]
df["speech_identifier"] = np.nan
df

,id,sitzung,wahlperiode,in_writing,speaker,speaker_cleaned,speaker_fp,sequence,text,filename,type,speaker_party,previous_speaker_fp,new_speaker,speech_identifier
0,2324045,1,17,0,Volker Kauder (CDU/CSU),Volker Kauder,volker-kauder,48,"Herr Alterspräsident, ich schlage für die CDU/...",data/txt/17001.txt,speech,cducsu,NaN,True,NaN
1,2324060,1,17,0,Dr. Norbert Lammert (CDU/CSU),Dr. Norbert Lammert,norbert-lammert,63,"Herr Präsident, ich nehme die Wahl gerne an.",data/txt/17001.txt,speech,cducsu,volker-kauder,True,NaN
2,2324144,1,17,0,Gerda Hasselfeldt (CDU/CSU),Gerda Hasselfeldt,gerda-hasselfeldt,147,"Herr Präsident, ich nehme die Wahl gerne an un...",data/txt/17001.txt,speech,cducsu,norbert-lammert,True,NaN
3,2324148,1,17,0,Dr. h. c. Wolfgang Thierse (SPD),Dr. h. c. Wolfgang Thierse,wolfgang-thierse,151,"Ja, ich nehme die Wahl an.",data/txt/17001.txt,speech,spd,gerda-hasselfeldt,True,NaN
4,2324152,1,17,0,Dr. Hermann Otto Solms (FDP),Dr. Hermann Otto Solms,hermann-otto-solms,155,Ich bedanke mich. Ich nehme die Wahl gerne an.,data/txt/17001.txt,speech,fdp,wolfgang-thierse,True,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
238643,2916173,108,18,0,Martin Patzelt (CDU/CSU),Martin Patzelt,martin-patzelt,359,Meine sehr verehrten Besucher! Meine Damen und...,data/txt/18108.txt,speech,cducsu,tom-koenigs,True,NaN
238644,2916179,108,18,0,Dr. Ute Finckh-Krämer (SPD),Dr. Ute Finckh-Krämer,ute-finckh-kramer,365,Frau Präsidentin! Liebe Kolleginnen und Kolleg...,data/txt/18108.txt,speech,spd,martin-patzelt,True,NaN
238645,2916185,108,18,0,Thorsten Frei (CDU/CSU),Thorsten Frei,thorsten-frei,371,Frau Präsidentin! Liebe Kolleginnen und Kolleg...,data/txt/18108.txt,speech,cducsu,ute-finckh-kramer,True,NaN
238646,2916187,108,18,0,Thorsten Frei (CDU/CSU),Thorsten Frei,thorsten-frei,373,"damit es möglich wird, den Verfassungsprozess ...",data/txt/18108.txt,speech,cducsu,thorsten-frei,False,NaN


In [9]:
%%time
speech_identifier = int(0)
for index, row in df.iterrows():
    if row["new_speaker"]:
        speech_identifier += 1
    df.at[index, "speech_identifier"] = speech_identifier

CPU times: user 18.7 s, sys: 89.6 ms, total: 18.8 s
Wall time: 19.1 s


In [10]:
df = (
    df.groupby(["speaker_fp", "speech_identifier"], sort=False)
    .agg(
        {
            "id": "count",
            "sitzung": "first",
            "wahlperiode": "first",
            "speaker": "first",
            "speaker_cleaned": "first",
            "sequence": min,
            "text": " ".join,
            "filename": "first",
            "type": "first",
            "speaker_party": "first",
        }
    )
    .reset_index()
)

In [11]:
df.to_pickle("data/plpr.pkl")

### All Text
To model topics, metainformation for speeches is not relevant. Everything in the text column can be glued together for that purpose.

In [12]:
path_alltext = Path("data/plpr_alltext.txt")

df["text"].to_csv(path_alltext, sep=" ", index=False, header=False)
preview_lines(path_alltext, N=13)

['"Herr Alterspräsident, ich schlage für die CDU/CSU-Bundestagsfraktion den Kollegen Dr. Norbert Lammert vor."\n',
 '"Herr Präsident, ich nehme die Wahl gerne an."\n',
 '"Herr Präsident, ich nehme die Wahl gerne an und bedanke mich herzlich für das Vertrauen."\n',
 '"Ja, ich nehme die Wahl an."\n',
 '"Ich bedanke mich. Ich nehme die Wahl gerne an."\n',
 '"Ich nehme die Wahl gerne an und freue mich auf die weitere Zusammenarbeit mit allen Kolleginnen und Kollegen."\n',
 '"Ich nehme die Wahl an und bedanke mich sehr herzlich."\n',
 '"Gemäß Artikel 63 Absatz 1 des Grundgesetzes für die Bundesrepublik Deutschland schlage ich dem Deutschen Bundestag vor, Frau Dr. Angela Merkel zur Bundeskanzlerin zu wählen. Eine besondere Überraschung scheint dieser Vorschlag nicht auszulösen. Ich bitte nun um Aufmerksamkeit für einige Hinweise zu unserem Wahlverfahren. Zu dieser Wahl sind die Stimmen der Mehrheit der Mitglieder des Bundestages, also mindestens 312 Stimmen, erforderlich. Nach unserer Geschä